# ClickHouse Analytics

This notebook demonstrates how to use ClickHouse for trace storage and analytics.

## Prerequisites

```bash
pip install neon-sdk[clickhouse]
```

You'll need a running ClickHouse instance. Start one with Docker:

```bash
docker run -d --name clickhouse -p 8123:8123 clickhouse/clickhouse-server
```

In [ ]:
# Install if needed
# !pip install neon-sdk[clickhouse]

## 1. Client Setup

In [ ]:
from neon_sdk.clickhouse import NeonClickHouseClient, ClickHouseConfig

# Create client
client = NeonClickHouseClient(ClickHouseConfig(
    host="localhost",
    port=8123,
    database="neon",
))

print("ClickHouse client created")
print(f"Connected to: {client.config.host}:{client.config.port}")

## 2. Inserting Traces

In [ ]:
from neon_sdk.clickhouse import TraceRecord, SpanRecord, ScoreRecord
from datetime import datetime, timedelta
import uuid

# Generate sample traces
project_id = "demo-project"
traces = []
spans = []
scores = []

for i in range(10):
    trace_id = str(uuid.uuid4())
    timestamp = datetime.now() - timedelta(hours=i)
    
    # Create trace
    traces.append(TraceRecord(
        trace_id=trace_id,
        project_id=project_id,
        name=f"agent-run-{i}",
        status="ok" if i % 3 != 0 else "error",
        timestamp=timestamp,
        duration_ms=1000 + (i * 100),
        input={"query": f"Query {i}"},
        output={"response": f"Response {i}"},
        metadata={"version": "1.0"},
    ))
    
    # Create spans for this trace
    spans.append(SpanRecord(
        span_id=str(uuid.uuid4()),
        trace_id=trace_id,
        parent_span_id=None,
        project_id=project_id,
        name="llm-call",
        span_type="generation",
        status="ok",
        start_time=timestamp,
        end_time=timestamp + timedelta(milliseconds=500),
        duration_ms=500,
        input={"prompt": "Hello"},
        output={"text": "Hi there!"},
        metadata={"model": "gpt-4"},
    ))
    
    # Create score
    scores.append(ScoreRecord(
        score_id=str(uuid.uuid4()),
        trace_id=trace_id,
        project_id=project_id,
        name="quality",
        value=0.8 + (i * 0.02),
        scorer_name="response_quality",
        reason="Good response",
        timestamp=timestamp,
    ))

print(f"Generated {len(traces)} traces, {len(spans)} spans, {len(scores)} scores")

# Insert data (uncomment when ClickHouse is running)
# client.insert_traces(traces)
# client.insert_spans(spans)
# client.insert_scores(scores)
# print("Data inserted!")

## 3. Querying Traces

In [ ]:
# Query traces (uncomment when ClickHouse is running)
# traces = client.query_traces(
#     project_id=project_id,
#     limit=10,
# )
# 
# print(f"Found {len(traces)} traces")
# for trace in traces:
#     print(f"  {trace.name}: {trace.status} ({trace.duration_ms}ms)")

# Simulated output
print("Query would return traces like:")
print("  agent-run-0: ok (1000ms)")
print("  agent-run-1: ok (1100ms)")
print("  agent-run-2: ok (1200ms)")
print("  agent-run-3: error (1300ms)")

## 4. Filtering Traces

In [ ]:
# Filter by status
# error_traces = client.query_traces(
#     project_id=project_id,
#     status="error",
#     limit=10,
# )

# Filter by date range
# recent_traces = client.query_traces(
#     project_id=project_id,
#     start_date="2024-01-01",
#     end_date="2024-01-31",
# )

# Filter by name pattern
# agent_traces = client.query_traces(
#     project_id=project_id,
#     name_pattern="agent-*",
# )

print("Filtering examples:")
print("  - By status: status='error'")
print("  - By date: start_date='2024-01-01', end_date='2024-01-31'")
print("  - By name: name_pattern='agent-*'")

## 5. Get Trace with Spans

In [ ]:
# Get full trace with spans
# result = client.get_trace_with_spans(project_id, trace_id)
# 
# trace = result["trace"]
# spans = result["spans"]
# 
# print(f"Trace: {trace.name}")
# print(f"Status: {trace.status}")
# print(f"Duration: {trace.duration_ms}ms")
# print(f"\nSpans ({len(spans)}):")
# for span in spans:
#     print(f"  - {span.name} ({span.span_type}): {span.duration_ms}ms")

print("Full trace retrieval shows:")
print("  Trace: agent-run-0")
print("  Status: ok")
print("  Duration: 1000ms")
print("  Spans (3):")
print("    - llm-call (generation): 500ms")
print("    - search (tool): 300ms")
print("    - postprocess (span): 200ms")

## 6. Dashboard Analytics

In [ ]:
# Get dashboard summary
# summary = client.get_dashboard_summary(
#     project_id=project_id,
#     start_date="2024-01-01",
#     end_date="2024-01-31",
# )
# 
# print(f"Dashboard Summary")
# print(f"=================")
# print(f"Total traces: {summary.total_traces}")
# print(f"Success rate: {100 - summary.error_rate:.1f}%")
# print(f"Avg duration: {summary.avg_duration_ms:.0f}ms")
# print(f"P95 duration: {summary.p95_duration_ms:.0f}ms")
# print(f"Total tokens: {summary.total_tokens}")

print("Dashboard Summary (example)")
print("===========================")
print("Total traces: 1,234")
print("Success rate: 95.2%")
print("Avg duration: 1,500ms")
print("P95 duration: 3,200ms")
print("Total tokens: 456,789")

## 7. Daily Statistics

In [ ]:
# Get daily stats
# daily = client.get_daily_stats(
#     project_id=project_id,
#     start_date="2024-01-01",
#     end_date="2024-01-07",
# )
# 
# print("Daily Statistics")
# print("================")
# for day in daily:
#     print(f"{day.date}: {day.trace_count} traces, {day.error_rate:.1f}% errors")

print("Daily Statistics (example)")
print("==========================")
print("2024-01-01: 156 traces, 3.2% errors")
print("2024-01-02: 189 traces, 4.1% errors")
print("2024-01-03: 142 traces, 2.8% errors")
print("2024-01-04: 201 traces, 5.0% errors")
print("2024-01-05: 178 traces, 3.9% errors")

## 8. Score Trends

In [ ]:
# Get score trends
# trends = client.get_score_trends(
#     project_id=project_id,
#     start_date="2024-01-01",
#     end_date="2024-01-31",
# )
# 
# print("Score Trends")
# print("============")
# for trend in trends:
#     print(f"{trend.scorer_name}:")
#     print(f"  Avg: {trend.avg_score:.2f}")
#     print(f"  Min: {trend.min_score:.2f}")
#     print(f"  Max: {trend.max_score:.2f}")

print("Score Trends (example)")
print("======================")
print("response_quality:")
print("  Avg: 0.85")
print("  Min: 0.45")
print("  Max: 0.98")
print("")
print("helpfulness:")
print("  Avg: 0.78")
print("  Min: 0.32")
print("  Max: 0.95")

## 9. Custom Queries

In [ ]:
# Execute custom SQL query
# result = client.execute_query(
#     """
#     SELECT
#         name,
#         count() as count,
#         avg(duration_ms) as avg_duration
#     FROM traces
#     WHERE project_id = {project_id:String}
#       AND timestamp >= {start:DateTime}
#     GROUP BY name
#     ORDER BY count DESC
#     LIMIT 10
#     """,
#     parameters={
#         "project_id": project_id,
#         "start": "2024-01-01 00:00:00",
#     }
# )
# 
# for row in result:
#     print(f"{row['name']}: {row['count']} traces, {row['avg_duration']:.0f}ms avg")

print("Custom Query Results (example)")
print("==============================")
print("customer-support: 456 traces, 1,200ms avg")
print("document-analysis: 234 traces, 2,100ms avg")
print("code-review: 189 traces, 3,500ms avg")

## 10. Visualization

In [ ]:
# Example: Create a simple chart with the data
import matplotlib.pyplot as plt

# Sample data (would come from get_daily_stats)
dates = ["Jan 1", "Jan 2", "Jan 3", "Jan 4", "Jan 5"]
trace_counts = [156, 189, 142, 201, 178]
error_rates = [3.2, 4.1, 2.8, 5.0, 3.9]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Trace counts
ax1.bar(dates, trace_counts, color='steelblue')
ax1.set_xlabel('Date')
ax1.set_ylabel('Trace Count')
ax1.set_title('Daily Trace Volume')

# Error rates
ax2.plot(dates, error_rates, marker='o', color='coral')
ax2.set_xlabel('Date')
ax2.set_ylabel('Error Rate (%)')
ax2.set_title('Daily Error Rate')
ax2.set_ylim(0, 10)

plt.tight_layout()
plt.show()

## Next Steps

- [04_temporal_workflows.ipynb](04_temporal_workflows.ipynb) - Durable workflow execution
- [Documentation](https://neon-sdk.readthedocs.io/en/latest/guides/clickhouse.html)